## Problem

**Based on the user input, we need to do some task**

```
if geom_type == 'circle': draw a circle
elif geom_type == 'square': draw a square
elif ...
```

# Step 1

## Monolithic Code

In [2]:
def draw(geom_type):
    if geom_type == 'circle':
        print("Drawing a circle")
    elif geom_type == 'square':
        print("Drawing a square")
    else:
        ...

In [4]:
# CLIENT CODE
geom_type = 'square'
draw(geom_type)

Drawing a square


# Step 2

## Towards Modular Code With Classes

In [5]:
from abc import ABC, abstractmethod

# Parent Class
class Shape(ABC):
    @abstractmethod
    def draw(self):
        pass
    
    @abstractmethod
    def area(self):
        pass

In [6]:
from math import pi

In [7]:
# Child Class
class Circle(Shape):
    def __init__(self,**kwargs):
        self.radius = kwargs['radius']
        
    def draw(self):
        return "Drawing a circle"
        
    @property
    def area(self):
        return pi * self.radius ** 2

class Square(Shape):
    def __init__(self,**kwargs):
        self.side = kwargs['side']
        
    def draw(self):
        return "Drawing a square"
        
    @property
    def area(self):
        return self.side ** 2


In [9]:
# CLIENT CODE
geom_type = 'circle'
radius = 2
side = 2

if geom_type == 'circle':
    geom = Circle(radius=radius)
elif geom_type == 'square':
    geom = Square(side=side)
else:
    ...
print(f'{geom.draw()} of area {geom.area}')

Drawing a circle of area 12.566370614359172


# Step 3.1 - Introducing Composition

### Prob 1) Class Hierarchy introduces Coupling (is-A relation)

In [10]:
# Child Class
class Circle(Shape):
    def __init__(self,**kwargs):
        self.radius = kwargs['radius']
        
    def draw(self):
        return "Drawing a circle"
        
    @property
    def area(self):
        return pi * self.radius ** 2

class Square(Shape):
    def __init__(self,**kwargs):
        self.side = kwargs['side']
        
    def draw(self):
        return "Drawing a square"
        
    @property
    def area(self):
        return self.side ** 2

# Class Hierarchy
class CircleWithHole(Circle):
    def __init__(self,**kwargs):
        super().__init__(radius=kwargs['radius'])
        self.hole_radius = kwargs['hole_radius']
        
    def draw(self):
        return "Drawing a circle with hole"
        
    @property
    def area(self):
        return super().area - pi * self.hole_radius ** 2

In [11]:
# CLIENT CODE
geom_type = 'circleWithHole'
radius = 2
side = 2
hole_radius = 1

if geom_type == 'circle':
    geom = Circle(radius=radius)
elif geom_type == 'square':
    geom = Square(side=side)
elif geom_type == 'circleWithHole':
    geom = CircleWithHole(radius=radius,hole_radius=hole_radius)
else:
    ...
geom

In [12]:
print(f'{geom.draw()} of area {geom.area}')

Drawing a circle with hole of area 9.42477796076938


### Sol 1) Removing coupling through Composition (has-A relation)

In [13]:
from abc import ABC, abstractmethod

# Parent Class
class Shape(ABC):
    
    @abstractmethod
    def area(self):
        pass

# Child Class
class Circle(Shape):
    def __init__(self,**kwargs):
        self.radius = kwargs['radius']
        
    def __repr__(self):
        return f'Circle(r={self.radius})'
        
    @property
    def area(self):
        return pi * self.radius ** 2

class Square(Shape):
    def __init__(self,**kwargs):
        self.side = kwargs['side']
        
    def __repr__(self):
        return f'Square(a={self.side})'
        
    @property
    def area(self):
        return self.side ** 2

In [15]:
from dataclasses import dataclass, field

@dataclass
class CustomShape:
    base_shape: Shape
    hole_shape: Shape

    @property
    def area(self):
        return self.base_shape.area - self.hole_shape.area

In [18]:
# CLIENT CODE
geom_type = 'square'
radius = 2
side = 2
hole_radius = 1

if geom_type == 'circle':
    base_shape = Circle(radius=radius)
elif geom_type == 'square':
    base_shape = Square(side=side)
    
hole_shape = Circle(radius=hole_radius)
geom = CustomShape(base_shape=base_shape,hole_shape=hole_shape)
geom

CustomShape(base_shape=Square(a=2), hole_shape=Circle(r=1))

In [19]:
print(f'Area = {geom.area}')

Area = 0.8584073464102069


# Step 3.2 - Introducing Factory

### Prob 2) Client Code has to change for adding new shapes

In [20]:
from abc import ABC, abstractmethod

# Parent Class
class Shape(ABC):
    
    @abstractmethod
    def area(self):
        pass

# Child Class
@dataclass
class Circle(Shape):
    def __init__(self,**kwargs):
        self.radius = kwargs['radius']
        
    def __repr__(self):
        return f'Circle(r={self.radius})'
        
    @property
    def area(self):
        return pi * self.radius ** 2

@dataclass
class Square(Shape):
    def __init__(self,**kwargs):
        self.side = kwargs['side']
        
    def __repr__(self):
        return f'Square(a={self.side})'
    
    @property
    def area(self):
        return self.side ** 2
    
@dataclass  
class Hexagon(Shape):
    def __init__(self,**kwargs):
        self.side = kwargs['side']
        
    def __repr__(self):
        return f'Hexagon(a={self.side})'
    
    @property
    def area(self):
        factor = 0.5 *(3 ** 1.5)
        return factor * self.side ** 2

In [21]:
# CLIENT CODE
geom_type = 'hexagon'
radius = 2
side = 2
hole_radius = 1

if geom_type == 'circle':
    geom = Circle(radius=radius)
elif geom_type == 'square':
    geom = Square(side=side)
elif geom_type == 'hexagon':
    geom = Hexagon(side=side)
else:
    ...


In [22]:
geom

Hexagon(a=2)

In [23]:
print(f'Area = {geom.area}')

Area = 10.392304845413264


### Sol 2) - Separating Creation From Use - Factory Pattern

In [24]:
# Separates Creation from Use
# Based on a particular type of input, we want to create instance and process
class ShapeFactory:
    def get_shape(geom_type,**kwargs):
        if geom_type == 'circle':
            return Circle(radius=kwargs['radius'])
        elif geom_type == 'square':
            geom = Square(side=kwargs['side'])
        elif geom_type == 'hexagon':
            geom = Hexagon(side=kwargs['side'])
        elif geom_type == 'circleWithHole':
            geom = CustomShape(
                base_shape=Circle(radius=kwargs['radius']),
                hole_shape=Circle(radius=kwargs['hole_radius']))
        elif geom_type == 'squareWithHole':
            geom = CustomShape(
                base_shape=Square(side=kwargs['side']),
                hole_shape=Circle(radius=kwargs['hole_radius']))
        else:
            ...
        return geom

In [28]:
# CLIENT CODE
geom_type = 'circleWithHole'
radius = 2
side = 2
hole_radius = 1

kwargs = {
    'radius':radius,
    'side':side,
    'hole_radius':hole_radius
}

geom = ShapeFactory.get_shape(geom_type,**kwargs)
geom

CustomShape(base_shape=Circle(r=2), hole_shape=Circle(r=1))

In [29]:
geom.area

9.42477796076938

# Step 4 - Factory in Production

In [30]:
class ShapesFactory:
    """
    Factory class to register new shapes
    """    
    registry = {}
    @classmethod
    def register_shape(cls,geom_type:str):
        def inner(wrapped_cls):
            cls.registry[geom_type] = wrapped_cls
            return wrapped_cls
        return inner
    
    @classmethod
    def get_shape(cls,geom_type:str,**kwargs):
        return cls.registry[geom_type](**kwargs)

In [ ]:
{
    'circle':Circle(radius),
    'square':Square(side),
}

In [56]:
from abc import ABC, abstractmethod

@dataclass
class RegularShape(ABC):
    
    @abstractmethod
    def area(self):
        pass


@dataclass
class CustomShape:
    base_shape: Shape
    hole_shape: Shape

    @property
    def area(self):
        return self.base_shape.area - self.hole_shape.area
    
@dataclass
@ShapesFactory.register_shape('circle')
class Circle(RegularShape):
    
    def __init__(self,**kwargs):
        self.radius = kwargs['radius']
        
    def __repr__(self):
        return f'Circle(r={self.radius})'
    
    @property
    def area(self):
        # from math import pi
        return pi * self.radius ** 2
        
@dataclass     
@ShapesFactory.register_shape('square')
class Square(RegularShape):
    
    def __init__(self,**kwargs):
        self.side = kwargs['side']
        
    def __repr__(self):
        return f'Square(a={self.side})'
    
    @property
    def area(self):
        return self.side ** 2

@dataclass        
@ShapesFactory.register_shape('hexagon')
class Hexagon(RegularShape):
    def __init__(self,**kwargs):
        self.side = kwargs['side']
        
    def __repr__(self):
        return f'Hexagon(a={self.side})'
    
    @property
    def area(self):
        factor = 0.5 *(3 ** 1.5)
        return factor * self.side ** 2
    
@dataclass
@ShapesFactory.register_shape('circleWithHole')
class CircleWithHole(CustomShape):
    def __init__(self,**kwargs):
        self.base_shape = Circle(radius=kwargs['radius'])
        self.hole_shape = Circle(radius=kwargs['hole_radius'])
        
    def __repr__(self):
        return f"CircleWithHole(r={self.base_shape.radius},r'={self.hole_shape.radius})"
    
@dataclass
@ShapesFactory.register_shape('squareWithHole')
class SquareWithHole(CustomShape):
    def __init__(self,**kwargs):
        self.base_shape = Square(side=kwargs['side'])
        self.hole_shape = Circle(radius=kwargs['hole_radius'])
        
    def __repr__(self):
        return f"SquareWithHole(a={self.base_shape.side},r'={self.hole_shape.radius})"
    

# Implementing New Regular Shape 
@dataclass        
@ShapesFactory.register_shape('rectangle')
class Rectangle(RegularShape):
    def __init__(self,**kwargs):
        self.length = kwargs['length']
        self.width = kwargs['width']
        
    def __repr__(self):
        return f'Rectangle(l={self.length}, b={self.width})'
    
    @property
    def area(self):
        return self.length * self.width
    
# Implementing Custom Shapes by just extending the class and implementing
@dataclass
@ShapesFactory.register_shape('hexagonWithHole')
class HexagonWithHole(CustomShape):
    def __init__(self,**kwargs):
        self.base_shape = Hexagon(side=kwargs['side'])
        self.hole_shape = Circle(radius=kwargs['hole_radius'])
        
    def __repr__(self):
        return f"HexagonWithHole({self.base_shape},{self.hole_shape})"
    

# New features can be implemented by just extending the class and implementing
@dataclass
@ShapesFactory.register_shape('rectangleWithHole')
class RectangleWithHole(CustomShape):
    def __init__(self,**kwargs):
        self.base_shape = Rectangle(length=kwargs['length'],width=kwargs['width'])
        self.hole_shape = Circle(radius=kwargs['hole_radius'])
        
    def __repr__(self):
        return f"RectangleWithHole({self.base_shape},{self.hole_shape})"

In [32]:
# CLIENT CODE
geom_type = 'square'
radius = 2
side = 2
hole_radius = 1

kwargs = {
    'radius':radius,
    'side':side,
    'hole_radius':hole_radius
}

geom = ShapesFactory.get_shape(geom_type,**kwargs)
print(f'{geom}, area = {geom.area}')

Square(a=2), area = 4


In [33]:
# CLIENT CODE
geom_type = 'circle'
radius = 2
side = 2
hole_radius = 1

kwargs = {
    'radius':radius,
    'side':side,
    'hole_radius':hole_radius
}

geom = ShapesFactory.get_shape(geom_type,**kwargs)
print(f'{geom}, area = {geom.area}')

Circle(r=2), area = 12.566370614359172


In [34]:
# CLIENT CODE
geom_type = 'circleWithHole'
radius = 2
side = 2
hole_radius = 1

kwargs = {
    'radius':radius,
    'side':side,
    'hole_radius':hole_radius
}

geom = ShapesFactory.get_shape(geom_type,**kwargs)
print(f'{geom}, area = {geom.area}')

CircleWithHole(r=2,r'=1), area = 9.42477796076938


In [35]:
# CLIENT CODE
geom_type = 'squareWithHole'
radius = 2
side = 2
hole_radius = 1

kwargs = {
    'radius':radius,
    'side':side,
    'hole_radius':hole_radius
}

geom = ShapesFactory.get_shape(geom_type,**kwargs)
print(f'{geom}, area = {geom.area}')

SquareWithHole(a=2,r'=1), area = 0.8584073464102069


In [58]:
# CLIENT CODE
geom_type = 'hexagonWithHole'
radius = 2
side = 2
hole_radius = 1

kwargs = {
    'radius':radius,
    'side':side,
    'hole_radius':hole_radius
}

geom = ShapesFactory.get_shape(geom_type,**kwargs)
print(f'{geom}, area = {geom.area}')

HexagonWithHole(Hexagon(a=2),Circle(r=1)), area = 7.250712191823471


In [57]:
# CLIENT CODE
geom_type = 'rectangleWithHole'
length = 3
width = 2
hole_radius = 1

kwargs = {
    'radius':radius,
    'length':length,
    'width':width,
    'hole_radius':hole_radius
}

geom = ShapesFactory.get_shape(geom_type,**kwargs)
print(f'{geom}, area = {geom.area}')

RectangleWithHole(Rectangle(l=3, b=2),Circle(r=1)), area = 2.858407346410207
